# Glioma-Immune-Glucose Steady States (Revised Model)
This notebook computes steady states for a revised version of the Sturrock et al. (2015) model,
incorporating both anti-oncogenic (e.g., CD8+ T cells) and pro-oncogenic (e.g., M2 macrophages) immune populations.

**Primary source:** Sturrock et al., *J. Theoretical Biology* (2015): [DOI: 10.1016/j.jtbi.2015.06.003](https://doi.org/10.1016/j.jtbi.2015.06.003)

In [ ]:
# Required Libraries
import numpy as np
from scipy.optimize import root
import pandas as pd
from itertools import product


## Model Equations
Each equation corresponds to a steady-state condition (d/dt = 0) for tumor cells (T), anti-immune cells (I_-),
pro-immune cells (I_+), brain glucose (σ_brain), and serum glucose (σ_serum).

In [ ]:
def steady_states(vars, params):
    T, I_minus, I_plus, sigma_brain, sigma_serum = vars
    (
        alpha_T, K_T, d_T, d_TI, gamma_TI,
        alpha_sigma, d_Tsigma, d_sigma1, alpha_s_minus, alpha_s_plus,
        nu, mu, d_I, d_I_plus, alpha_TI_minus, alpha_TI_plus,
        d_TT_minus, d_TT_plus, d_sigma2, F
    ) = params

    eq1 = alpha_T * sigma_brain * T * (1 - T / K_T) - d_T * T - d_TI * T * I_minus + gamma_TI * T * I_plus
    eq2 = alpha_sigma * (sigma_serum - sigma_brain) - d_Tsigma * T * sigma_brain - d_sigma1 * sigma_brain \
          - alpha_s_minus * (nu + I_minus) * sigma_brain - alpha_s_plus * (mu + I_plus) * sigma_brain
    eq3 = alpha_s_minus * (nu + I_minus) + alpha_TI_minus * T * I_minus - d_I * I_minus - d_TT_minus * T * I_minus
    eq4 = alpha_s_plus * (mu + I_plus) + alpha_TI_plus * T * I_plus - d_I_plus * I_plus - d_TT_plus * T * I_plus
    eq5 = alpha_sigma * (sigma_brain - sigma_serum) + F - d_sigma2 * sigma_serum

    return [eq1, eq2, eq3, eq4, eq5]

## Parameters and Sources
- Most parameters from Sturrock et al. (2015)
- Others assumed based on plausible biological values from literature (see inline comments)

In [ ]:
# Parameter values
params = [
    1.575,    # alpha_T: Tumor growth rate [Giese et al. 1996]
    2.0,      # K_T: Tumor carrying capacity [Matzavinos & Chaplain 2003]
    1e-4,     # d_T: Natural tumor decay
    0.072,    # d_TI: Tumor death by I_- [Sturrock et al.]
    0.03,     # gamma_TI: Tumor promotion by I_+ [assumed]
    20.0,     # alpha_sigma: Glucose transfer rate [Sturrock et al.]
    1.0,      # d_Tsigma: Tumor glucose consumption [Sturrock et al.]
    0.01,     # d_sigma1: Brain glucose use [Man et al. 2007]
    0.7,      # alpha_s_minus: I_- recruitment [Sturrock et al.]
    0.7,      # alpha_s_plus: I_+ recruitment [assumed similar]
    0.7,      # nu: Baseline for I_- [Sturrock et al.]
    0.3,      # mu: Baseline for I_+ [assumed lower]
    0.01,     # d_I: I_- decay [assumed]
    0.01,     # d_I_plus: I_+ decay [assumed]
    3e-4,     # alpha_TI_minus: Tumor-driven I_- [Sturrock et al.]
    3e-4,     # alpha_TI_plus: Tumor-driven I_+ [assumed]
    0.05,     # d_TT_minus: Tumor suppressing I_- [assumed]
    0.01,     # d_TT_plus: Tumor exhausting I_+ [assumed]
    0.01,     # d_sigma2: Serum glucose use [Man et al. 2007]
    1.6e-3    # F: Glucose intake [Man et al. 2007]
]

In [ ]:
# Root solver to enforce biological realism
def bounded_root_solver(initial_guess, params):
    def eqs(vars): return steady_states(vars, params)
    sol = root(eqs, initial_guess, method='hybr')
    if sol.success and np.all(sol.x >= -1e-6):  # Allow tiny negatives
        return np.round(sol.x, 6)
    else:
        return None

In [ ]:
# Scan multiple initial guesses for biologically meaningful solutions
guesses = [
    [0.01, 0.1, 0.1, 1e-4, 1.5e-3],
    [1.0, 0.5, 0.3, 5e-4, 2e-3],
    [0.5, 0.2, 0.2, 1e-3, 1e-3]
]

results = []
for guess in guesses:
    sol = bounded_root_solver(guess, params)
    if sol is not None:
        results.append(sol)

steady_df = pd.DataFrame(results, columns=["T*", "I_-*", "I_+*", "σ_brain*", "σ_serum*"])
steady_df